# Homework 3 Question 2: Car Rental
A small car rental company has a fleet of 94 vehicles distributed among its 10
agencies. The location of every agency is given by its geographical coordinates x and y in a grid based
on miles. We assume that the road distance between agencies is approximately 1.3 times the Euclidean
distance (as the crow flies). The following table indicates the coordinates of all agencies, the number
of cars required the next morning, and the stock of cars in the evening preceding this day.

Agency number |  1|  2|  3|  4|  5|  6|  7|  8|  9|  10
 --- |--- |--- |--- |--- |--- |--- |--- |--- |--- |--- 
x-coordinate | 0 | 20|  18|  30|  35|  33|  5|  5|  11|  2
y-coordinate | 0|  20|  10|  12|  0|  25|  27|  10|  0|  15
Required cars|  10|  6|  8|  11|  9|  7|  15|  7|  9|  12
Cars present | 8|  13|  4|  8|  12|  2|  14|  11|  15|  7

Supposing the cost for transporting a car is $0.50 per mile, the movements of cars that allow
the company to re-establish the required numbers of cars at all agencies, minimizing the total cost
incurred for transport.

## Problem Data 

In [1]:
# Helper Function
function agencyEuclDistance(ax, ay, bx, by)
    return 1.3*(sqrt((ax-bx)^2 + (ay-by)^2))
end;

In [2]:
using NamedArrays

# agency list
agency = [1,2,3,4,5,6,7,8,9,10]

# required cars per agency
required = Dict(zip(agency, [10 6 8 11 9 7 15 7 9 12]))

# present cars per agency
present =  Dict(zip(agency, [8 13 4 8 12 2 14 11 15 7]))

# location (x,y) tuples per agency
location = Dict(zip(agency,[( 0 ,  0)
                            ( 20,  20)
                            ( 18,  10)
                            ( 30,  12)
                            ( 35,   0)
                            ( 33,  25)
                            (  5,  27)
                            (  5,  10)
                            ( 11,   0)
                            (  2,  15)]))

# cost per mile for transportation
costPerMile = 0.5

# Arrays to store agency to agency cost (agency, agency)
costRaw = reshape([costPerMile*agencyEuclDistance(location[a][1],location[a][2],
            location[b][1],location[b][2])
    for a in agency for b in agency],length(agency), length(agency))
cost = NamedArray(costRaw, (agency,agency), ("Agency", "Agency"))
;

## Problem Model 

In [3]:
using JuMP

m = Model()

# Variables for edge cost from agency to agency
@variable(m, x[agency,agency] >= 0)

# Constraint to limit the number of cars an agency can transfer to the number of cars present
@constraint(m, supply[a in agency], sum(x[a,b] for b in agency) <= present[a])
            
# Constraint to impose the required number of cars at the end at each agency
@constraint(m, demand[b in agency], sum(x[a,b] for a in agency) == required[b])

# Objective of minimizing cost per car transfer between agencies
@objective(m, Min, sum(cost[a,b]*x[a,b] for a in agency for b in agency))
;

In [4]:
status = solve(m)
println("Minimum cost to reistablish demands: \$",getobjectivevalue(m))

# nicely formatted solution
raw = Int[getvalue(x[i,j]) for i in agency, j in agency]
raw = hcat(raw,sum(raw,2))
raw = vcat(raw,sum(raw,1))

solution = NamedArray(raw,([1,2,3,4,5,6,7,8,9,10,"Required"],
        [1,2,3,4,5,6,7,8,9,10,"Pres"]),("Src","Dest"))
println(solution)

Minimum cost to reistablish demands: $152.63901632295628
11×11 Named Array{Int64,2}
Src ╲ Dest │    1     2     3     4     5     6     7     8     9    10  Pres
───────────┼─────────────────────────────────────────────────────────────────
1          │    8     0     0     0     0     0     0     0     0     0     8
2          │    0     6     1     0     0     5     1     0     0     0    13
3          │    0     0     4     0     0     0     0     0     0     0     4
4          │    0     0     0     8     0     0     0     0     0     0     8
5          │    0     0     0     3     9     0     0     0     0     0    12
6          │    0     0     0     0     0     2     0     0     0     0     2
7          │    0     0     0     0     0     0    14     0     0     0    14
8          │    0     0     0     0     0     0     0     7     0     4    11
9          │    2     0     3     0     0     0     0     0     9     1    15
10         │    0     0     0     0     0     0     0     

The car transfers from source agency to destination agency is as shown above. The rightmost column shows the cars initially present at all source agencies, while the bottom most row shows the required cars at each destination agency. The values in these rows and columns is as per the requirement of the question.